In [1]:
import hashlib
import hmac
import math
import os
import random
random.seed(5)

# Feistel Cipher

In cryptography, a Feistel cipher is a symmetric structure used in the construction of block ciphers, named after the German IBM cryptographer Horst Feistel; it is also commonly known as a Feistel network. A large set of block ciphers use the scheme, including the Data Encryption Standard. The Feistel structure has the advantage that encryption and decryption operations are very similar, even identical in some cases, requiring only a reversal of the key schedule. Therefore the size of the code or circuitry required to implement such a cipher is nearly halved. Feistel construction is iterative in nature which makes implementing the cryptosystem in hardware easier. 

## Encryption Process

The encryption process uses the Feistel structure consisting multiple rounds of processing of the plaintext, each round consisting of a “substitution” step followed by a permutation step.

Feistel Structure is shown in the following illustration:

<img src='images/feistel_structure.jpg' >

### XOR Functions
In this notebook, we assume that the data are in **binary** format. Implement the XOR function to xor two bytes.


In [2]:
# xor two strings
def xor(b1, b2):
    # create empty string for the output
    ret = ''
    
    # create empty list 
    result = []
    
    # go through each bit of the bytes and xor them
    for bit1, bit2 in zip(b1, b2):
        #xor two bits
        xor_bit = bit1 ^ bit2
        
        # convert it into bytes format
        xor_bytes = bytes([xor_bit])
                          
        # append it to the result
        result.append(xor_bytes)

        
        
    # convert result list into binary string using b''.join()    
        ret =  b''.join(result)
    
    # return the string
    return ret


In [3]:
s1 = bytes('A'.encode())
s2 = bytes('B'.encode())
s1_xor_s2 = xor(s1,s2)
print('s1 xor s2 xor s2: ', xor(s1_xor_s2, s2).decode())
print('s1 xor s2: ', s1_xor_s2)

s1 xor s2 xor s2:  A
s1 xor s2:  b'\x03'


** Expected Output: **
<table>
  <tr>
      <td> s1 xor s2 xor s2 </td>
      <td> A </td>
  </tr>
  <tr>
      <td> s1 xor s2 </td>
      <td> b'\x03' </td>
  </tr>
</table>

### Create plain text blocks
Implement `create_block` to create message blocks from an input source. the length of each block is specified by the parameter *block_size*.   

In [4]:
def create_block(source, block_size=8):
    # define empty block_list
    block_list = []
    
    # get the length of the source
    source_length = len(source)
    
    # go through the 
    for i in range(0, source_length - block_size, block_size):
        # get the block of the source 
        source_block = source[i:i+block_size]
        
        # append to the block list 
        block_list.append(source_block)
        
    
    # get the index for the last block
    last_block_index = math.floor(source_length/block_size) *  block_size
    
    # add the remaining source to the last block
    last_block = source[last_block_index:]
    
    # get the length of the last block
    last_block_len = len(last_block)
    
    # if the length of the last block is not equal to the block size
    # pad spaces to the end of the block
    for i in range(last_block_len, block_size):
        last_block+= bytes(" ".encode())
    
    
    # add the last block to the block list
    block_list.append(last_block)
   
    # return the block list
    return block_list


In [5]:
source = bytes("Hello World!".encode())
block_list = create_block(source, block_size=8)
print('block_list: ', block_list)

block_list:  [b'Hello Wo', b'rld!    ']


** Expected Output: **
<table>
  <tr>
      <td> block_list: </td>
      <td> [b'Hello Wo', b'rld!    '] </td>
    </tr>
</table>

### Round Function
Implement `scramble()` that uses the SHA algorithm to perform has on the input message and the key. We use the *hmac* and *hashlib* Python libraries.

In [6]:
def scramble(msg, key):
    # create a hmac sha1 
    h = hmac.new(key, msg, hashlib.sha1)
    # Return first 8 bytes of the calculated hmac sha1 hash
    return h.digest()[:8]


In [7]:
x = bytes('Hello World!'.encode())
i = 1
k = bytes('C'.encode())
F = scramble(x,k)
print('F = ', F)

F =  b'I\x0cf\x11\xfe\xffx4'


** Expected Output: **
<table>
  <tr>
      <td> F = </td>
      <td> b'I\x0cf\x11\xfe\xffx4' </td>
    </tr>
</table>

### Initial Key
We generate 16 random cryptographic secure bytes as key using `os.urandom` function.

In [8]:
def key_256():
    # encode unicode
    key = os.urandom(16)
    return key

In [9]:
print('key: ', key_256())

key:  b'>n<R\xf2~\xfb\xf7\x00)b \xdf\x0e|('


### Round keys
Implement `subkeygen` to pull the specific key for each round of the Feistel Cipher. 

In [10]:
def subkeygen(initial_key, i):
    # encode the unicode
    keys = ([bytes([k]) for k in initial_key])[:16]
    return keys[i]

In [11]:
initial_key = key_256()
k_15 = subkeygen(initial_key, 15)
print('key_15: ', k_15)

key_15:  b'%'


### Feistel round procedure
Implement `feistel_round()` to implement the procedure for each round in Feistel Algorithm:

- assign right input side to the left output
- apply round function (F) on the right input
- XOR the result with Left input and assign it to Right output
- return left and right output

In [12]:
def feistel_round(L_in, R_in, subkey):
    # assign right input side to the left output
    L_out = R_in
    
    # apply scramble function on the right input
    R_scrambled = scramble(R_in, subkey)
    
    # XOR R_scrambled with Left input and assign it to Right output
    R_out = xor(R_scrambled, L_in)
    
    # return left and right output
    return L_out, R_out
    
    
    

In [13]:
L_in = bytes('DE7F'.encode())
R_in = bytes('03A6'.encode()) 
round_num = 15
subkey = subkeygen(initial_key, round_num)
L_out, R_out = feistel_round(L_in, R_in, subkey)

print("Encryption:")
print("L_out: ", L_out)
print("R_out: ", R_out)

L_out, R_out = feistel_round(R_out, L_out, subkey)

print("Decryption:")
print("L_out: ", L_out)
print("R_out: ", R_out)



Encryption:
L_out:  b'03A6'
R_out:  b'1P\x19G'
Decryption:
L_out:  b'03A6'
R_out:  b'DE7F'


** Expected Output: **
 Note: it is possible that the R_out is different in your output 
<table>
  <tr>
      <td> Encryption </td>
      <td> L_out:  b'03A6'</td>
      <td> R_out:  b'\xe0\xfd\xeb5' </td>
  </tr>
  
  <tr>
      <td> Decryption </td>
      <td> L_out:  b'03A6'</td>
      <td> R_out:  b'DE7F' </td>
  </tr>

</table>

### Feistel Cipher
Implement `feistel_cipher()` to implement Feistel Algorithm to perform encryption and decryption:

1. Split the message_block two left and right side
2. Go through each round 
3. Get subkey using initial key and round iteration
4. Perform Feistel round procedure
5. Update the left and right input for the next round 
6. Set the left output of the last round as the the right side (vice versa)

In [14]:
def feistel_cipher(key_initial,message_block, round_num, mode='encoding'):
    
    # create empty string for return ciphertext
    output_block = ' '
    
    
    # split the message_block two left and right side
    # calculate the split_index
    block_len = len(message_block)
    split_index = int(block_len/2)
    
    
    # get the intial Left and right sides   
    L_in = message_block[:split_index]
    R_in = message_block[split_index:block_len]
    
    
    
    # go through each round 
    for i in range(1,round_num):
        
        
        # set the round_iteration number based on the encoding or decoding mode
        round_iteration = i if mode == 'encoding' else round_num - i  
        
        
        # generate subkey using initial key and round iteration
        subkey = subkeygen(key_initial, round_iteration)
        
        # perform feistel round
        L_out, R_out = feistel_round(L_in, R_in, subkey)
        
        
        # update the left and right input for the next round 
        L_in = L_out
        R_in = R_out
        
        #print(L_in)
        #print(R_in)
        #print(" ")
        
       
        
    
    # set the left output of the last round as the the right side (vice versa)
    L = R_in
    R = L_in
    
    # concatinate the final left and right   
    output_block = L + R
    
    # return the block
    return output_block

In [15]:
message = bytes('DE7F03A6'.encode())
round_num = 16
# generate a 256 bit key based on key
key_initial = key_256()

cipher_message = feistel_cipher(key_initial,message, round_num)

print("cipher_message: ",cipher_message)

plain_message = feistel_cipher(key_initial,cipher_message, round_num, mode='decoding')

print("plain_message: " , plain_message)


cipher_message:  b']^m\x0e\xca\x93\xddR'
plain_message:  b'DE7F03A6'


** Expected Output: **
<table>
  <tr>
      <td> cipher_message </td>
      <td>b'\x8e\x91m\xebZ\xa8\x11\x1d'</td>
  </tr>
  
  <tr>
      <td> plain_message </td>
      <td> b'DE7F03A6'</td>
  </tr>

</table>

### Encode and Decode a file content using Feistel Cipher
Complete the following procedure to:

1. open a file to read as binary
2. create plain text message blocks from the file content
3. encode the message blocks using Feistel encoder
4. perform encoding for each block
5. perform decoding for each block
6. compare the original message with the decoded message to verify that the encoding/decoding is passed.

In [16]:
# set file location
# Update this location with your system path
file_location = '/home/abel/Documents/154xb.txt'

# open a file to read as binary
f = open(file_location,'rb')

# read the file content
file_content = f.read()

# close the file
f.close()
print('file_content_length: ', len(file_content))

# create plain text message blocks from the file content
# set block_size to be 8
block_list = create_block(file_content, 8)
print('number of message blocks: ', len(block_list))

# encode the message blocks using feistel encoder
# set the round number to 16
round_num = 16

# generate a 256 bit key based on key
key_initial = key_256()

# perform encoding for each block
cipher_list = []
for plain_message in block_list:

    # perform feistel encryption
    cipher_message = feistel_cipher(key_initial, plain_message, round_num, mode='encoding')
    cipher_list.append(cipher_message)

# perform decoding for each block
decoded_block_list = []
for cipher_message in cipher_list:
    # perform feistel decryption
    decoded_message = feistel_cipher(key_initial,cipher_message, round_num, mode='decoding')
    decoded_block_list.append(decoded_message)

# compare the original message with the decoded one
passed = 0
for original, decoded in zip(block_list, decoded_block_list):
    if original == decoded:
        passed += 1

if passed == len(block_list):
    print('Decoding PASSED :)')
else:
    print('Decoding FAILED :(')
    


file_content_length:  8693
number of message blocks:  1087
Decoding PASSED :)


** Expected Output: **
<table>
  <tr>
      <td> file_content_length </td>
      <td> 8693</td>
  </tr>
  
  <tr>
      <td> number of message blocks </td>
      <td> 1087</td>
  </tr> 
  <tr>
    <td> Decoding PASSED :) </td>
  </tr>

</table>